# SCIBERT in HuggingFace

In [8]:
# run this script to import the variables and settings from ipynb_util.py
%run ipynb_util.py

In [5]:
from datasets import load_dataset

# load the dataset
# note: if you don't have the data in the folder, use the download-data.sh script
dataset = load_dataset("json", data_files=str(DATA_DIR_PATH / "task1.jsonl"))
dataset = dataset["train"].train_test_split(test_size=0.3, seed=SEED)

example = dataset["train"][0]
print("Example instance:\t", example)

labels = set(dataset["train"]["SDG"])
# identity because labels are already ids and vice-versa
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels

Example instance:	 {'ID': 'oai:www.zora.uzh.ch:124422', 'TITLE': 'How sexual selection can drive the evolution of costly sperm ornamentation', 'ABSTRACT': 'Post-copulatory sexual selection (PSS), fuelled by female promiscuity, is credited with the rapid evolution of sperm quality traits across diverse taxa1. Yet, our understanding of the adaptive significance of sperm ornaments and the cryptic female preferences driving their evolution is extremely limited. Here we review the evolutionary allometry of exaggerated sexual traits (for example, antlers, horns, tail feathers, mandibles and dewlaps), show that the giant sperm of some Drosophila species are possibly the most extreme ornaments in all of nature and demonstrate how their existence challenges theories explaining the intensity of sexual selection, mating-system evolution and the fundamental nature of sex differences. We also combine quantitative genetic analyses of interacting sex-specific traits in D. melanogaster with comparativ

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}

In [10]:
from transformers import AutoTokenizer

# base model
HF_MODEL_NAME = "allenai/scibert_scivocab_uncased"
# final model
MODEL_NAME = f"{HF_MODEL_NAME}-ft-task1"

tokenizer = AutoTokenizer.from_pretrained(HF_MODEL_NAME)

def preprocess_data(instances):
    # take a batch of titles and abstracts and concat them
    titles = instances["TITLE"]
    abstracts = instances["ABSTRACT"]
    texts = [f"{title} {abstract}" for title, abstract in zip(titles, abstracts)]
    # encode
    encoding = tokenizer(texts, padding="max_length", truncation=True, max_length=512, return_tensors="pt")

    # add labels
    encoding["label"] = torch.tensor([label2id[label] for label in instances["SDG"]])

    return encoding

encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=dataset["train"].column_names)
encoded_dataset.set_format("torch")

In [4]:
example = encoded_dataset["train"][0]
print("Example instance:\t", example)

tokenizer.decode(example["input_ids"])

Example instance:	 {'input_ids': tensor([  102,   539,  5471,  2516,   300,  7021,   111,  2696,   131, 13773,
         6619, 21457,  4972,   150,  1422,   579,  1756,  7655,  5471,  2516,
          145, 24343,   546,   422,  8086,   810,   214,  3672,  1699,  2432,
         7627,   422,   165, 22249,   190,   111,  2857,  2696,   131,  6619,
         1671,  7378,  2186,  6555, 12026, 30130,   205,  3481,   422,   580,
         2934,   131,   111,  4640,  4150,   131,  6619, 21457, 11874,   137,
          111,  8846,   141,  3672,  7637,  7290,   547,  2696,   165,  6343,
         2379,   205,  1530,   185,  1579,   111,  6644, 15031, 29868,   131,
        24320,   224,  5471,  7378,   145,   168,  1143,   422,  4129, 21480,
          422, 13835, 30113,   422,  5533,   588,  5838, 30113,   422, 26510,
        30113,   137,   223, 18367,  1970,   546,   422,   405,   198,   111,
        15278,  6619,   131,   693, 10760,  1578,   220,  5779,   111,   755,
         7424, 21457, 11874,   

'[CLS] how sexual selection can drive the evolution of costly sperm ornamentation post - copulatory sexual selection ( pss ), fuelled by female promiscuity, is credited with the rapid evolution of sperm quality traits across diverse taxa1. yet, our understanding of the adaptive significance of sperm ornaments and the cryptic female preferences driving their evolution is extremely limited. here we review the evolutionary allometry of exaggerated sexual traits ( for example, antlers, horns, tail feathers, mandibles and dewlaps ), show that the giant sperm of some drosophila species are possibly the most extreme ornaments in all of nature and demonstrate how their existence challenges theories explaining the intensity of sexual selection, mating - system evolution and the fundamental nature of sex differences. we also combine quantitative genetic analyses of interacting sex - specific traits in d. melanogaster with comparative analyses of the condition dependence of male and female reprod

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    HF_MODEL_NAME,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at allenai/scibert_scivocab_uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import TrainingArguments, Trainer, EvalPrediction

BATCH_SIZE = 8
METRIC_NAME = "accuracy"

args = TrainingArguments(
    f"{CHECKPOINT_PATH}/{MODEL_NAME}",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    #save_steps=10,
    #eval_steps=10,
    #logging_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=20,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model=METRIC_NAME,
    seed=SEED
)

def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    accuracy = accuracy_score(labels, pred.predictions.argmax(-1))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred.predictions.argmax(-1), average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [7]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,2.294022,0.348837,0.121687,0.348837,0.180433
2,No log,1.983403,0.387597,0.319233,0.387597,0.257885
3,No log,1.756778,0.457364,0.391972,0.457364,0.384278
4,No log,1.666955,0.511628,0.415087,0.511628,0.448823
5,No log,1.752029,0.527132,0.440522,0.527132,0.463918
6,No log,1.663559,0.519380,0.462906,0.519380,0.482689
7,No log,1.692489,0.527132,0.527017,0.527132,0.504697
8,No log,1.768125,0.542636,0.565891,0.542636,0.528349
9,No log,1.789269,0.519380,0.636693,0.519380,0.530452
10,No log,1.972066,0.534884,0.633583,0.534884,0.546809


/home/dvdblk/dev/_thesis/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dvdblk/dev/_thesis/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/dvdblk/dev/_thesis/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

TrainOutput(global_step=760, training_loss=0.5132972466318231, metrics={'train_runtime': 597.7961, 'train_samples_per_second': 10.07, 'train_steps_per_second': 1.271, 'total_flos': 1584156096552960.0, 'train_loss': 0.5132972466318231, 'epoch': 20.0})

## Evaluation

In [8]:
trainer.evaluate()

/home/dvdblk/dev/_thesis/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 2.0742533206939697,
 'eval_accuracy': 0.5503875968992248,
 'eval_precision': 0.6207630338957288,
 'eval_recall': 0.5503875968992248,
 'eval_f1': 0.5498825849667871,
 'eval_runtime': 4.3855,
 'eval_samples_per_second': 29.415,
 'eval_steps_per_second': 3.876,
 'epoch': 20.0}

## Inference

In [16]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

text = "This is a test text about climate change."

MODEL_PATH = f"{CHECKPOINT_PATH}/{MODEL_NAME}/checkpoint-532"
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    num_labels=len(labels),
    id2label={i: label for i, label in enumerate(labels)},
    label2id={label: i for i, label in enumerate(labels)}
).to("cuda")
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model.eval()
encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(model.device) for k, v in encoding.items()}

outputs = model(**encoding)

In [17]:
logits = outputs.logits
logits.shape

torch.Size([1, 18])

In [25]:
softmax = torch.nn.Softmax()
probs = softmax(logits.squeeze().cpu())

# for multi-label classification, we need to threshold the probabilities
"""
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
predicted_labels = [id2label(i) for i in np.where(predictions == 1)]
print(predicted_labels)
"""
# for single-label classification, we can take the argmax
print(probs)
predictions = probs.argmax(-1)
predicted_label = id2label[predictions.int().item()]
print(predicted_label)

tensor([1.4732e-03, 8.4256e-04, 1.2375e-03, 1.2201e-03, 1.0317e-03, 1.0760e-03,
        1.3087e-03, 1.3920e-03, 2.3354e-03, 7.2098e-04, 6.6897e-04, 8.4898e-04,
        8.5957e-04, 9.7841e-01, 1.9451e-03, 1.9424e-03, 1.5772e-03, 1.1073e-03],
       grad_fn=<SoftmaxBackward0>)
13


In [ ]:
import gc

def my_reset(*varnames):
    """
    Resets global variables and majority of CUDA memory. Only works in Jupyter.

    varnames are what you want to keep
    """
    try:
        del model
    except NameError:
        pass
    gc.collect()
    with torch.no_grad():
        torch.cuda.ipc_collect()
        torch.cuda.empty_cache()

    globals_ = globals()
    to_save = {v: globals_[v] for v in varnames}
    to_save['my_reset'] = my_reset  # lets keep this function by default
    del globals_
    get_ipython().magic("reset")
    globals().update(to_save)

# my_reset()

/tmp/ipykernel_27454/3741609671.py:22: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic("reset")


Nothing done.


### Investigating sklearn precision warning

In [ ]:
p = torch.ones((86, 18)).argmax(-1)
p[:10] = torch.Tensor([i for i in range(10)])
t = torch.ones(86, dtype=torch.long)
t[:18] = torch.Tensor([i for i in range(18)])

# no Warning
precision_recall_fscore_support(t, p, average="weighted", labels=[i for i in range(10)])

(0.9873459873459873, 0.1282051282051282, 0.12816755893678972, None)

In [ ]:
# Warning
precision_recall_fscore_support(t, p, average="weighted", labels=[i for i in range(11)])

/home/user/dbielik/msc-thesis/venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.9748479368732533, 0.12658227848101267, 0.12654518477303286, None)